In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.8 MB 4.9 MB/s 
     |████████████████████████████████| 181 kB 41.1 MB/s 
     |████████████████████████████████| 145 kB 30.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (GRU, LSTM, Dense, Dropout,
                                     SimpleRNN, TimeDistributed, Layer)
from tensorflow import expand_dims, reduce_sum, multiply, concat, split, nn, reduce_mean
from tensorflow import print as pr
from numpy import zeros


In [4]:
import time

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import tensorflow as tf
from matplotlib.font_manager import FontProperties



import time
from IPython.display import clear_output
from IPython.display import HTML as html_print
from IPython.display import display
import matplotlib
import numpy as np

In [5]:

def clean_files():
    base_file_name = "/content/drive/MyDrive/hi/lexicons/hi.translit.sampled."
    for filename_extension in ["train", "test", "dev"]:
        f = open("temp", mode='wt', encoding='utf-8')
        f.write("Hi\tEn\tLexicons\n")

        filename = base_file_name + filename_extension + ".tsv"
        data_file = open(filename, mode='rt', encoding='utf-8')
        for line in data_file.readlines():
            f.write(line)

        data = pd.read_table("temp")
        data.to_csv(filename_extension + ".csv")

clean_files()

In [6]:
def Encoding():
  #Train Test File Name
  Train_File_Name="/content/drive/MyDrive/hi/lexicons/hi.translit.sampled.train.tsv"
  Val_File_Name="/content/drive/MyDrive/hi/lexicons/hi.translit.sampled.dev.tsv"
  Test_File_Name="/content/drive/MyDrive/hi/lexicons/hi.translit.sampled.test.tsv"
  Enc_Tokens=0
  Dec_token=0
  Enc_Data=[]
  Dec_Data=[]
  Dec_Target_Data=[]

  Target=[]
  Input=[]
  input_words=[]
  target_words=[]
  inp_char=set()
  tar_char=set()
  TempFile=open('CombinedData.tsv',mode='wt')
  for filename in [Train_File_Name,Val_File_Name,Test_File_Name]:
    with open(filename,mode='rt') as Current_File:
      data=Current_File.read()
      data=data.split('\n')
      #TempFile.write(data)
      for words in data[:len(data)-1]:
        target,input,x=words.split("\t")
        target = "\t" + target + "\n"
        for char in input:
          inp_char.add(char)
        for char in target:
          tar_char.add(char)
        input_words.append(input)
        target_words.append(target)
     


  inp_char = sorted(list(inp_char)) + ["\n"]
  tar_char = sorted(list(tar_char)) + ["\n"]  
  Enc_Tokens_Count=len(inp_char)
  Dec_Tokens_Count=len(tar_char)
  max_len_word_encoder=max([len(txt) for txt in input_words])
  max_len_word_decoder=max([len(txt) for txt in target_words])

  input_token_index = dict([(char, i) for i, char in enumerate(inp_char)])
  target_token_index = dict([(char, i) for i, char in enumerate(tar_char)])
  Dec_Data=np.zeros((len(input_words),max_len_word_decoder,Dec_Tokens_Count),dtype="float32")
  Dec_Target_Data=np.zeros((len(input_words),max_len_word_decoder,Dec_Tokens_Count),dtype="float32")
  Enc_Data=np.zeros((len(input_words),max_len_word_encoder,Enc_Tokens_Count),dtype="float32")

  for index,(target,input) in enumerate((zip(target_words,input_words))):
    for i,char in enumerate(input):
      Enc_Data[index,i,input_token_index[char]]=1.0
    Enc_Data[index,i+1:,input_token_index["\n"]]=1.0
    for i,char in enumerate(target):
      Dec_Data[index,i,target_token_index[char]]=1.0
      if i>0:
        Dec_Target_Data[index,i-1,target_token_index[char]]=1.0
    Dec_Data[index,i+1:,target_token_index["\n"]]=1.0
    Dec_Target_Data[index,i:,target_token_index["\n"]]=1.0

  File_Length=[]
  Input_Encode={}
  Input_Decode={}
  Target_Decode={}
  Input_Text={}
  Target_Text ={}
  for filename in [Train_File_Name,Val_File_Name,Test_File_Name]:
    with open(filename,mode='rt') as Current_File:
      data=Current_File.readlines()
      File_Length.append(len(data))
  Current=0
  Data_Category=['Train','Val','Test']
  for file_length,data_name in zip(File_Length,Data_Category):
    Input_Encode[data_name]=Enc_Data[Current:Current+file_length,:,:]
    Input_Decode[data_name]=Dec_Data[Current:Current+file_length,:,:]
    Target_Decode[data_name]=Dec_Target_Data[Current:Current+file_length,:,:]
    Input_Text[data_name]=np.array(input_words[Current:Current+file_length])
    Target_Text[data_name]=np.array(target_words[Current:Current+file_length])
    Current+=file_length


            
  print("Number of samples:", len(input_words))
  print("Number of unique input tokens:", Enc_Tokens_Count)
  print("Number of unique output tokens:", Dec_Tokens_Count)
  print("Max sequence length for inputs:", max_len_word_encoder)
  print("Max sequence length for outputs:", max_len_word_decoder)
  print((Input_Encode['Test']).size)
  print(target_token_index)
  return Input_Encode,Input_Decode,Target_Decode,Input_Text,Target_Text,input_token_index,target_token_index,max_len_word_decoder,Dec_Tokens_Count,Enc_Tokens_Count
      
Input_Encode,Input_Decode,Target_Decode,Input_Text,Target_Text,input_token_index,target_token_index,max_len_word_decoder,Dec_Tokens_Count,Enc_Tokens_Count=Encoding()
print(target_token_index)

Number of samples: 53064
Number of unique input tokens: 27
Number of unique output tokens: 66
Max sequence length for inputs: 20
Max sequence length for outputs: 21
2431080
{'\t': 0, '\n': 65, 'ँ': 2, 'ं': 3, 'ः': 4, 'अ': 5, 'आ': 6, 'इ': 7, 'ई': 8, 'उ': 9, 'ऊ': 10, 'ऋ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'ङ': 21, 'च': 22, 'छ': 23, 'ज': 24, 'झ': 25, 'ञ': 26, 'ट': 27, 'ठ': 28, 'ड': 29, 'ढ': 30, 'ण': 31, 'त': 32, 'थ': 33, 'द': 34, 'ध': 35, 'न': 36, 'प': 37, 'फ': 38, 'ब': 39, 'भ': 40, 'म': 41, 'य': 42, 'र': 43, 'ल': 44, 'व': 45, 'श': 46, 'ष': 47, 'स': 48, 'ह': 49, '़': 50, 'ा': 51, 'ि': 52, 'ी': 53, 'ु': 54, 'ू': 55, 'ृ': 56, 'ॅ': 57, 'े': 58, 'ै': 59, 'ॉ': 60, 'ो': 61, 'ौ': 62, '्': 63, 'ॐ': 64}
{'\t': 0, '\n': 65, 'ँ': 2, 'ं': 3, 'ः': 4, 'अ': 5, 'आ': 6, 'इ': 7, 'ई': 8, 'उ': 9, 'ऊ': 10, 'ऋ': 11, 'ए': 12, 'ऐ': 13, 'ऑ': 14, 'ओ': 15, 'औ': 16, 'क': 17, 'ख': 18, 'ग': 19, 'घ': 20, 'ङ': 21, 'च': 22, 'छ': 23, 'ज': 24, 'झ': 25, 'ञ': 26, 'ट': 27, 'ठ

In [7]:
class BahdanauAttention(Layer):
  def __init__(self, units):
    super(BahdanauAttention,self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, query, values):
    if(len(query.shape)==2):
      query = expand_dims(query, 1)
      s1=self.W1(query)
      s2=self.W2(values)
      Value=self.V(nn.tanh(s1+s2))
      Att_W=nn.softmax(Value, axis=1)
      Context_VEC=Att_W * values
      Context_VEC=reduce_sum(Context_VEC, axis=1)
      return Context_VEC, Att_W
    else:
      query=expand_dims(query,2)
      values=expand_dims(values,1)
      s1 = self.W1(query)
      s2 = self.W2(values)
      tsum = s1+s2
      Value = self.V(nn.tanh(tsum))
      Att_W = nn.softmax(Value, axis=2)
      Context_VEC = Att_W * values
      Context_VEC = reduce_sum(Context_VEC, axis=2)
      return Context_VEC, Att_W

In [8]:
#Function That Creates Model for Encoder Decoder
def Gen_Model(Layer_Dim,Cell_Type,R_Dropout,dp,IfAttention,Attn_Shape,Encoder_TS,Decoder_TS,Encoder_VS,Decode_VS,Embedding_Size,Encoder_Layers,Decoder_Layers):
  CellType={'LSTM':LSTM,'RNN':SimpleRNN,'GRU':GRU}
  temp=1
  if Cell_Type=='LSTM':
    temp=2
  Layers=len(Layer_Dim)
  Encoder_Inputs=keras.Input(shape=(Encoder_TS,Encoder_VS))
  Decoder_Inputs=keras.Input(shape=(Decoder_TS,Decode_VS))
  last_layer_out=Encoder_Inputs
  Encoder_States=[]
  enc_states=[]
  Enc_Layers=[]

  Encoder_Info = keras.layers.Embedding(input_dim = Enc_Tokens_Count, output_dim = Embedding_Size, input_length = max_len_word_decoder)(Encoder_Inputs)
  Encoder_Layer_Dim = Layer_Dim* Encoder_Layers
  Decoeder_Layer_Dim  = Layer_Dim * Decoder_Layers

  for i in range(Layers):
    Encoder_Info=CellType[Cell_Type](Layer_Dim[i],recurrent_dropout=R_Dropout,dropout=dp,return_state=True,return_sequences=True)
    Encoder_Out=Encoder_Info(last_layer_out)
    enc_states,Encoder_Out=concat(list(Encoder_Out[1:]),axis=-1),Encoder_Out[0]
    last_layer_out=Encoder_Out
    Encoder_States.append(enc_states)
    Enc_Layers.append(Encoder_Info)

  Decoder_States=[]
  Dec_Layers=[]
  last_layer_out=Decoder_Inputs
  for i in range(Layers):
    Decoder_Info=CellType[Cell_Type](Layer_Dim[i],return_state=True,recurrent_dropout=R_Dropout,dropout=dp,return_sequences=True)
    Decoder_Out=Decoder_Info(last_layer_out,initial_state=split(Encoder_States[i],temp, -1))
    dec_states,Decoder_Out=(list(Decoder_Out[1:])),Decoder_Out[0]
    last_layer_out=Decoder_Out
    Decoder_States.append(dec_states)
    Dec_Layers.append(Decoder_Info)

  attn = BahdanauAttention(Attn_Shape)
  attn_context, attn_weights = attn(last_layer_out, Encoder_Out)
  Decoder_dropout=Dropout(dp)
  Decoder_dropout_out=Decoder_dropout(last_layer_out)

  Decoder_Dense=Dense(Decode_VS,activation='softmax')
  Decoder_Time_Distribution=TimeDistributed(Decoder_Dense)
  Decoder_Output=Decoder_Time_Distribution(Decoder_dropout_out)
  Final_Model=Model(inputs=[Encoder_Inputs,Decoder_Inputs],outputs=Decoder_Output)
  
#Work For EncoderModel
  Enc_input=Input(shape=(Encoder_TS, Encoder_VS))
  ENC_states=[]
  Last_layer_Out = Enc_input
  for i in range(Layers):
    ENC_Out=Enc_Layers[i](Last_layer_Out)
    Enc_Lstmstate,ENC_Out=concat(list(ENC_Out[1:]),axis=-1),ENC_Out[0]
    ENC_states.append(Enc_Lstmstate)
    Last_layer_Out = ENC_Out
    Encoder_Model=Model(Enc_input,[ENC_states,Last_layer_Out])
    Dec_Input=Input(shape=(1, Decode_VS))
    Dec_State_Lstm=[Input(shape=(temp*Layer_Dim[i],))for i in range(Layers)]
    Encoder_Out=Input(shape=(Encoder_TS,Layer_Dim[Layers-1]))
    prev_layer_out=Dec_Input
    DEc_States=[]
    for i in range(Layers):
        Decoder_LSTM_OUT=Dec_Layers[i](prev_layer_out,initial_state=split(Dec_State_Lstm[i],temp,axis=-1))
        DecoderL_State,Decoder_LSTM_OUT=concat(list(Decoder_LSTM_OUT[1:]),axis=-1),Decoder_LSTM_OUT[0]
        prev_layer_out=Decoder_LSTM_OUT
        DEc_States.append(DecoderL_State)
    attn_context, attn_weights = attn(prev_layer_out, Encoder_Out)
    Decoder_Dropout=Decoder_dropout(prev_layer_out)
    Dec_Output=TimeDistributed(Decoder_Dense)(Decoder_Dropout)
    Decoder_Model=Model([Dec_Input,Dec_State_Lstm, Encoder_Out],[Dec_Output,DEc_States])
  return Final_Model,Encoder_Model,Decoder_Model

In [ ]:
#Without Sweep
#Uncomment To execute
"""
cell_type = 'GRU'
layer_dimension = [512]
attention = False
attention_shape = 256
dropout = 0.2
recurrent_dropout = 0.1
Embedding_Size=1
Encoder_Layers=2
Decoder_Layers=3
full_model,Enc_Model,Dec_Model = Gen_Model(layer_dimension,cell_type,recurrent_dropout,dropout,attention,attention_shape,20, 21, 27, 66,Embedding_Size,Encoder_Layers,Decoder_Layers)
full_model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
full_model.fit(x=[Input_Encode["Train"][:1000000],Input_Decode["Train"][:1000000]],y=Target_Decode["Train"][:1000000],batch_size=128,epochs=10,validation_data=([Input_Encode["Val"],Input_Decode["Val"]],Target_Decode["Val"]))
full_model.save("Model"+"/train")
Enc_Model.save("Model"+"/Enc_Model")
Dec_Model.save("Model"+"/Dec_Model")"""

In [25]:
def decode_sequence(input_seqs, encoder_model, decoder_model):
  #Encoding as state vector
  Value_State, enc_out = encoder_model.predict(input_seqs)
  old_states_value = Value_State[:]
  TargetS=np.zeros((len(input_seqs),1,Dec_Tokens_Count))
  TargetS[:,0,target_token_index['\t']] = 1.
  Stop=False
  Sentence_Decoded=[""]*len(input_seqs)
  while not Stop:
    to_split=decoder_model.predict([TargetS,Value_State,enc_out])
    output_tokens,Value_State, attn_weights=to_split[0],list(to_split[1:-1]),to_split[-1]
    Sampled_Token=np.argmax(output_tokens, axis=-1)
    sampled_chars=[reverse_target_char_index[Sampled_Token[i][0]] for i in range(0,len(input_seqs))]
    for i in range(0, len(input_seqs)) :
      Sentence_Decoded[i]=Sentence_Decoded[i] + str(sampled_chars[i])
      if len(Sentence_Decoded[0])>max_len_word_decoder:
        Stop = True
      TargetS = np.zeros((len(input_seqs),1,Dec_Tokens_Count))
      for i in range(0,len(input_seqs)):
        TargetS[i,0,Sampled_Token[i]] =1.
  Sentence_Decoded=[seq[0:seq.find('\n')] for seq in Sentence_Decoded]
  #print(Sentence_Decoded)
  return Sentence_Decoded

In [29]:
import numpy as np
from tqdm import tqdm
from wandb.keras import WandbCallback
import wandb
import os

#Main Funciton That Creates Model And Find Predictions over VAlidation Data Set
def Main(config=None):
  with wandb.init(config=config):
    config=wandb.config
    wandb.run.name =f"cell_type_{config.cell_type}_layer_org_{config.layer_dimensions}_drpout_{(config.dropout)}_rec-drpout_{(config.recurrent_dropout)}_bs_{config.batch_size}_opt_{config.optimizer}"

    full_model,inc_model,inf_dec_model=Gen_Model(config.layer_dimensions,config.cell_type,config.recurrent_dropout,config.dropout,config.attention,config.attention_shape,20, 21, 27, 66,config.Embedding_Size,config.Encoder_Layers,config.Decoder_Layers)
    full_model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
    full_model.fit(x=[Input_Encode["Train"][:999999],Input_Decode["Train"][:999999]],y=Target_Decode["Train"][:999999],batch_size=128,epochs=1,validation_data=([Input_Encode["Val"],Input_Decode["Val"]], Target_Decode["Val"]),callbacks=[WandbCallback(save_model=False)])

    RTargetIndex=dict((i, char) for char,i in target_token_index.items())


    def decode_sequence(input_seqs, encoder_model, decoder_model):
        # Encode the input as state vectors.
        states_value, enc_out = encoder_model.predict(input_seqs)
        old_states_value = states_value[:]
        target_seq = np.zeros((len(input_seqs), 1, Dec_Tokens_Count))
        target_seq[:, 0, target_token_index['\t']] = 1.
        stop_condition = False
        decoded_sentence = [""] * len(input_seqs)
        while not stop_condition:
            to_split = decoder_model.predict([target_seq, states_value, enc_out])
            output_tokens,states_value, attn_weights = to_split[0], list(to_split[1:-1]), to_split[-1]
            sampled_token_index = np.argmax(output_tokens, axis = -1)
            sampled_chars = [RTargetIndex[sampled_token_index[i][0]] for i in range(0, len(input_seqs))]
            for i in range(0, len(input_seqs)) :
                decoded_sentence[i] = decoded_sentence[i] + str(sampled_chars[i])
            if len(decoded_sentence[0]) > max_len_word_decoder:
                stop_condition = True
            target_seq = np.zeros((len(input_seqs), 1, Dec_Tokens_Count))
            for i in range(0, len(input_seqs)) :
              target_seq[i, 0, sampled_token_index[i]] = 1.
        decoded_sentence = [seq[0:seq.find('\n')] for seq in decoded_sentence]
        return decoded_sentence

    input_seqs=Input_Encode["Val"]
    target_sents=Target_Text["Val"]
    input_texts=Input_Text["Val"]
    Acc_Val=0
    size_batch=64
    n=len(input_seqs)
    TableLog=[]
    for IndexSq in tqdm(range(0,n,size_batch)):
        input_seq=input_seqs[IndexSq:min(n,IndexSq + size_batch)]
        decoded_sentences=decode_sequence(input_seq, inc_model, inf_dec_model)
        target_sentences=[str(target_sents[i : i + 1][0][1:-1]) for i in range(IndexSq, min(n, IndexSq + size_batch))]
        for i in range(0,len(decoded_sentences)) :
          if(decoded_sentences[i]==target_sentences[i]):
              Acc_Val += 1
        if(IndexSq < size_batch):
            for i in range(IndexSq, min(n, IndexSq +size_batch)) :
              TableLog.append([input_texts[i], decoded_sentences[i-IndexSq],target_sentences[i-IndexSq]])
              print({f"input_{i}": input_texts[i],f"output_{i}":decoded_sentences[i-IndexSq],f"target_{i}":target_sentences[i-IndexSq]})

    wandb.log({"Validation Table":wandb.Table(data=TableLog,columns=["Input","Decoded","Target"])})
    Acc_Val/=n
    wandb.log({"val_avg_acc":Acc_Val})


In [30]:
Sconfig = {'name': 'Test-Sweep', 'method': 'grid'}
# Wandb default config
#Use The Sweep Config To change The following Hyperparameters
parameters_dict  = {
    "epochs": {"values":[10]},
    "batch_size": {"values":[64]},
    "layer_dimensions": {"values":[[64]]},
    "cell_type": {"values":['GRU']},
    "dropout": {"values":[0.1]},
    "recurrent_dropout": {"values":[0.2]},
    "optimizer": {"values":["adam"]},
    "attention": {"values":[True]},
    "attention_shape":{"values":[256]},
    "Embedding_Size":{"values":[32]},
    "Encoder_Layers":{"values":[2]},
    "Decoder_Layers":{"values":[2]},
    "Beam_Width":{"values":[1]},

}

Sconfig['parameters'] = parameters_dict

sweep_id = wandb.sweep(Sconfig, project = 'DL_Assignment3PartB')
wandb.agent(sweep_id, function=Main)  

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: i4nwc4e7
Sweep URL: https://wandb.ai/sahilb/DL_Assignment3PartB/sweeps/i4nwc4e7


wandb: Agent Starting Run: nhlezqp9 with config:
wandb: 	Beam_Width: 1
wandb: 	Decoder_Layers: 2
wandb: 	Embedding_Size: 32
wandb: 	Encoder_Layers: 2
wandb: 	attention: True
wandb: 	attention_shape: 256
wandb: 	batch_size: 64
wandb: 	cell_type: GRU
wandb: 	dropout: 0.1
wandb: 	epochs: 10
wandb: 	layer_dimensions: [64]
wandb: 	optimizer: adam
wandb: 	recurrent_dropout: 0.2
wandb: Currently logged in as: sahilb. Use `wandb login --relogin` to force relogin


 97/346 [=======>......................] - ETA: 22s - loss: 2.0420 - accuracy: 0.6484

wandb: Ctrl + C detected. Stopping sweep.


In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from random import sample
def Maps_Attention(input_word,heatmap_data,decoded):
    matx = []
    dec_inputs = []
    for data in heatmap_data:
        dec_ind, attn  = data[0], data[1]
        matx.append(attn.reshape(-1)[:len(input_word)])
        dec_inputs.append(dec_ind)
    Attention_Map = np.array(matx)
    fig, ax = plt.subplots()
    ax.imshow(Attention_Map)
    ax.set_yticks(np.arange(Attention_Map.shape[0]))
    ax.set_yticklabels( decoded, fontproperties = FontProperties(fname = "/content/Lohit-Devanagari.ttf"))
    ax.set_xticklabels([char for char in input_word])
    ax.tick_params(labelsize = 15)
    ax.tick_params(axis ='x',labelrotation=45)
    return fig

In [ ]:
#Best Model Test
from tqdm import tqdm
config_defaults = {
    "epochs": 10,
    "batch_size": 128,
    "layer_dimensions": [128, 128],
    "cell_type": "LSTM",
    "dropout": 0.1,
    "recurrent_dropout": 0.1,
    "optimizer": "adam",
    "attention": False,
    "attention_shape": 256
}

with wandb.init(config=config_defaults):
  config=wandb.config
  wandb.run.name = f"Best_Model"#f"cell_type_{config.cell_type}_layer_org_{config.layer_dimensions}_drpout_{int(config.dropout)}_rec-drpout_{int(config.recurrent_dropout)}_bs_{config.batch_size}_opt_{config.optimizer}"

  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

  Enc_Model = tf.keras.models.load_model("Model"+"/Enc_Model")
  Dec_Model = tf.keras.models.load_model("Model"+"/Dec_Model")

  input_seqs = Input_Encode["Test"]
  target_sents = Target_Text["Test"]
  input_texts = Input_Text["Test"]
  n = len(input_seqs)
  log_table = []
  test_acc = 0
  BATCH_SIZE = 64

  li=sample(range(n), 10)
  predictions_vanilla = open("Predictions" + "Attention-Best_Model", 'w')
  for seq_index in tqdm(range(0, n, BATCH_SIZE)):
      # Take one sequence (part of the training set)
      # for trying out decoding.
      
      input_seq = input_seqs[seq_index:min(n, seq_index + BATCH_SIZE)]
      decoded_sentences,heatmap = decode_sequence(
          input_seq, Enc_Model, Dec_Model)
      target_sentences = [str(target_sents[i : i + 1][0][1:-1]) for i in range(seq_index, min(n, seq_index + BATCH_SIZE))]
      print(input_texts[seq_index])

      for i in range(0, len(decoded_sentences)) :
        if(decoded_sentences[i] == target_sentences[i]):
            test_acc += 1
      for i in range(seq_index, min(n, seq_index + BATCH_SIZE)) :
          predictions_vanilla.write(input_texts[i] + " | " + decoded_sentences[i - seq_index] + " | " + target_sentences[i - seq_index] + '\n')
      if(seq_index < BATCH_SIZE):
          for i in range(seq_index, min(n, seq_index + BATCH_SIZE)) :
            Hmap=Maps_Attention(input_texts[i],heatmap,decoded_sentences[i-seq_index])
            wandb.log( {"heatmap_" + str(i): Hmap})
            log_table.append(
                [input_texts[i], decoded_sentences[i - seq_index], target_sentences[i - seq_index]])
            print({f"input_{i}": input_texts[i], f"output_{i}": decoded_sentences[i - seq_index],f"target_{i}": target_sentences[i - seq_index]})

  wandb.log({"Validation log table": wandb.Table(data=log_table,columns=["Input", "Prediction", "Target"])})
      

  #val_avg_edit_dist /= val_samples
  #val_acc /= val_samples
  #wandb.log({"val_avg_edit_dist": val_avg_edt_dist, "val_avg_acc": val_acc})

  print("Test accuracy is : ", test_acc / n)
  wandb.log({"Test accuracy" : test_acc / n})

In [ ]:
#Testing on Best Model Test


#Wandb Initialization
with wandb.init():
  wandb.run.name = f"Best_Model"
  Enc_Model =tf.keras.models.load_model("Model"+"/Enc_Model")
  Dec_Model =tf.keras.models.load_model("Model"+"/Dec_Model")
  input_seqs = Input_Encode["Test"]
  target_sents=Target_Text["Test"]
  input_texts=Input_Text["Test"]
  n=len(input_seqs)
  log_table=[]
  test_acc=0
  BATCH_SIZE=64

  predictions_vanilla = open("Predictions"+"Best_Model", 'w')
  for seq_index in tqdm(range(0,n,BATCH_SIZE)):
      input_seq = input_seqs[seq_index:min(n,seq_index+BATCH_SIZE)]
      Decoded_Data,heatmap = decode_sequence(input_seq,Enc_Model, Dec_Model)
      Target_Data=[str(target_sents[i : i + 1][0][1:-1]) for i in range(seq_index, min(n, seq_index + BATCH_SIZE))]
      edit_distances=[]
      for i in range(0,len(Decoded_Data)) :
        if(Decoded_Data[i]==Target_Data[i]):
            test_acc+=1
      for i in range(seq_index,min(n,seq_index+BATCH_SIZE)) :
          predictions_vanilla.write(input_texts[i] +" | "+Decoded_Data[i -seq_index] + " | " +Target_Data[i-seq_index] +'\n')
      if(seq_index < BATCH_SIZE):
          for i in range(seq_index,min(n, seq_index + BATCH_SIZE)) :
            Hmap=Maps_Attention(input_texts[i],heatmap,decoded_sentences[i-seq_index])
            wandb.log( {"heatmap_" + str(i): Hmap})
            log_table.append([input_texts[i],Decoded_Data[i-seq_index], Target_Data[i-seq_index]])
            print({f"input_{i}":input_texts[i],f"output_{i}": Decoded_Data[i-seq_index],f"target_{i}":Target_Data[i-seq_index]})

  wandb.log({"Validation Table":wandb.Table(data=log_table,columns=["Input", "Prediction", "Target"])})
  print("Test accuracy is : ", test_acc/n)
  wandb.log({"Test accuracy":test_acc/n})

References:

https://predictablynoisy.com/matplotlib/tutorials/colors/colormaps.html

https://rdrr.io/cran/htmltools/man/html_print.html

In [10]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))
#Gradient Plottting
def Gradients_Plot(Map_List):
    norow=len(Map_List)
    temp= 0.35+0.15+(norow+(norow-1)*0.1)*0.22
    fig, axs = plt.subplots(nrows=norow + 1,figsize=(6.4,temp))
    fig.subplots_adjust(top=1-0.35/temp,bottom=0.15/temp,left=0.2,right=0.99)
    for ax, name in zip(axs, Map_List):
        ax.imshow(gradient, aspect='auto',cmap=plt.get_cmap(name))
        ax.text(-0.01,0.5,name,va='center',ha='right',fontsize=10,transform=ax.transAxes)
    for ax in axs:
        ax.set_axis_off()
    return plt

def Style(s,Color="YlGn"):
	if s == ' ':
		return "<text style=color:#000;padding-left:10px;background-color:{}> </text>".format(Color,s)
	else:
		return "<text style=color:#000;background-color:{}>{} </text>".format(Color,s)

#To show Color
def Show_Color(word):
	display(html_print(''.join([Style(first, Color=second) for first,second in word])))

#Function to return Color Encoding 
def Color_fun(weight,Color="YlGn"):
  cmap=matplotlib.cm.get_cmap(Color)
  rgba=cmap(weight)
  color=matplotlib.colors.rgb2hex(rgba)
  return color




In [11]:
#To Visualize the Connectivity
#Run the Above and Below Cell T generate the Visualization
def Visualize_Map(Weight,Decoded,Word,Color="YlGn"):
  for c in range(len(Decoded)):
    clear_output(wait=True)
    Gradients_Plot(["YlGn"]).show()
    print("Input Word : ",Word)
    print("Decoded text : ",Decoded)
    text_colors=[]
    temp = []
    print("Vizualized Input:-")
    for d in range(1,len(Word)+1):
      temp.append(Weight[c][d])
    for d in range(1,len(Word)+1):
      text = (Word[d-1],Color_fun(Weight[c][d]))
      text_colors.append(text)
    Show_Color(text_colors)
    print("Decoder Output : ",Decoded[c],end="")
    time.sleep(2)

#Sample weight and Input predicted from the above results
Weight=[[0,0.7,0.3,0,0,0],[0,0,0.2,0,0,0],[0,0,0,0.8,0,0],[0,0,0,0,0.6,0],[0,0,0,0,0,0.9]]
Decoded="मारना"
Word="marna"
Visualize_Map(Weight,Decoded,Word)

Input Word :  marna
Decoded text :  मारना
Vizualized Input:-


Decoder Output :  ा